# Get data

In [1]:
from awpy import Demo

path = ".demos/"

match = "faze-vs-cloud-9-m1-mirage.dem"

dem = Demo(path + match)

In [2]:
for event_name, event in dem.events.items():
    print(f"{event_name}: {event.shape[0]} rows x {event.shape[1]} columns")

round_prestart: 19 rows x 16 columns
player_spawn: 190 rows x 36 columns
cs_pre_restart: 19 rows x 16 columns
bomb_begindefuse: 6 rows x 37 columns
hltv_chase: 395 rows x 23 columns
player_blind: 216 rows x 58 columns
bomb_beginplant: 11 rows x 37 columns
round_officially_ended: 36 rows x 16 columns
inferno_startburn: 80 rows x 40 columns
inferno_expire: 80 rows x 40 columns
bomb_defused: 3 rows x 38 columns
round_announce_match_point: 1 rows x 16 columns
chat_message: 8 rows x 37 columns
hltv_versioninfo: 1 rows x 17 columns
hltv_fixed: 111 rows x 24 columns
flashbang_detonate: 110 rows x 40 columns
player_hurt: 479 rows x 62 columns
bomb_dropped: 48 rows x 37 columns
decoy_detonate: 2 rows x 40 columns
other_death: 85 rows x 47 columns
round_announce_match_start: 1 rows x 16 columns
item_pickup: 1256 rows x 39 columns
player_death: 140 rows x 95 columns
weapon_zoom: 157 rows x 36 columns
begin_new_match: 1 rows x 16 columns
player_team: 10 rows x 41 columns
bomb_exploded: 2 rows x 38

# Rounds and Outcomes

* total number of rounds played
* win rate of each team (CT and T)
* Rounds where the bomb was planted but not defused

## total rounds

In [6]:
dem.rounds.shape[0]

19

There were 19 rounds played

## win rate of each team

In [3]:
if list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[0] == "CT":
    first_half_CT = list(dem.kills[['victim_team_name', 'victim_team_clan_name']].iloc[0, :])[1]
    first_half_T = dem.kills.loc[dem.kills['victim_team_clan_name'] != first_half_CT]['victim_team_clan_name'].iloc[0,]

    second_half_CT, second_half_T = first_half_T, first_half_CT

print(f"CT 1st half: {first_half_CT}\n T 1st half: {first_half_T}\nCT 2nd half: {second_half_CT}\n T 2nd half: {second_half_T}")

CT 1st half: FaZe Clan
 T 1st half: Cloud 9
CT 2nd half: Cloud 9
 T 2nd half: FaZe Clan


In [4]:
import pandas as pd
import numpy as np


dem.rounds['winning_team'] = np.select(
    [
        (dem.rounds['round'].between(1, 12, inclusive='both')) & (dem.rounds['winner'] == "CT"), 
        (dem.rounds['round'].between(1, 12, inclusive='both')) & (dem.rounds['winner'] == "T"),
        (dem.rounds['round'].between(13, 24, inclusive='both')) & (dem.rounds['winner'] == "CT"),
        (dem.rounds['round'].between(13, 24, inclusive='both')) & (dem.rounds['winner'] == "T")
    ], 
    [
        first_half_CT, 
        first_half_T,
        second_half_CT,
        second_half_T
        
    ], 
    default='Unknown'
)


dem.rounds['half'] = np.select(
    [
        (dem.rounds['round'].between(1, 12, inclusive='both')) , 
        (dem.rounds['round'].between(13, 24, inclusive='both')) ,
    ], 
    [
        "First", 
        "Second",
        
    ], 
    default='Overtime'
)


In [5]:
summary_series = dem.rounds.groupby(['winning_team', 'winner', 'half'])['round'].count()

sum_table = pd.DataFrame(summary_series).sort_values(['winning_team', 'half'])

# sum_table['new_col'] = sum_table['round']/13

sum_table['proportion'] = np.select(
    [
        sum_table.index.get_level_values('half') == "First",
        sum_table.index.get_level_values('half') == "Second"
    ],
    [
        round(sum_table['round']/12, 4),
        round(sum_table['round']/7, 4)
    ],
    default='err'
)

sum_table

round proportion
winning_team winner half                    
Cloud 9      T      First       4     0.3333
             CT     Second      2     0.2857
FaZe Clan    CT     First       8     0.6667
             T      Second      5     0.7143

## Rounds where bomb was planted and not defused

In [6]:
rounds_bomb_table = dem.rounds.set_index('round').join(dem.bomb.set_index('round'))

planted_not_defused_rounds = rounds_bomb_table[(rounds_bomb_table['event'] == 'planted')]

planted_not_defused_rounds[planted_not_defused_rounds['reason'] != 'bomb_defused']

,start,freeze_end,end,official_end,winner,reason,bomb_plant,winning_team,half,tick,event,site,X,Y,Z,ticks_since_round_start,ticks_since_freeze_time_end,ticks_since_bomb_plant,clock
round,,,,,,,,,,,,,,,,,,,
4,22506,23786,29265,29713,T,ct_killed,27115,Cloud 9,First,27115.0,planted,BombsiteA,-278.550476,-2157.540527,-173.722778,4609,3329,0,00:40
9,67639,71090,76198,76646,T,ct_killed,74168,Cloud 9,First,74168.0,planted,BombsiteA,-283.385559,-2158.823730,-173.823608,6529,3078,0,00:40
10,76646,77926,85727,86175,T,bomb_exploded,83103,Cloud 9,First,83103.0,planted,BombsiteB,-1987.031250,398.769440,-159.968750,6457,5177,0,00:40
12,95263,96543,104802,108290,T,ct_killed,102824,Cloud 9,First,102824.0,planted,BombsiteB,-1969.045532,450.031250,-159.968750,7561,6281,0,00:40
15,123931,125211,129936,130384,T,ct_killed,129852,FaZe Clan,Second,129852.0,planted,BombsiteB,-1969.044067,450.031494,-159.968750,5921,4641,0,00:40
16,130384,131664,135914,136362,T,ct_killed,135712,FaZe Clan,Second,135712.0,planted,BombsiteB,-1987.032349,404.700836,-159.968750,5328,4048,0,00:40
18,140074,143680,152032,152480,T,ct_killed,149993,FaZe Clan,Second,149993.0,planted,BombsiteB,-2027.634155,421.893127,-159.968750,9919,6313,0,00:40


In [7]:
dem.ticks[(dem.ticks['game_time'].isin(dem.events['bomb_beginplant']['game_time'])) & (dem.ticks['round'] == 10)]

dem.ticks[dem.ticks['tick'] == 82907]

,inventory,accuracy_penalty,zoom_lvl,is_bomb_planted,ping,health,has_defuser,has_helmet,flash_duration,last_place_name,...,X,pitch,yaw,Y,Z,game_time,tick,steamid,name,round
829000,[],NaN,NaN,False,0,0,False,False,0.0,BombsiteB,...,-2267.947266,-0.201187,72.199738,346.285461,-164.954590,4225.609375,82907,76561198201620490,broky,10
829001,"[bayonet, Glock-18, AK-47, Smoke Grenade]",0.00641,0.0,False,0,100,False,True,0.0,BombsiteB,...,-1605.112549,0.792053,42.750885,18.742342,-168.037964,4225.609375,82907,76561198045433304,Ax1Le,10
829002,"[knife_butterfly, Desert Eagle, AK-47, Molotov]",0.00000,NaN,False,0,68,True,True,0.0,CTSpawn,...,-1192.983154,3.207672,-178.929169,-931.775574,-167.968750,4225.609375,82907,76561197997351207,rain,10
829003,[],NaN,NaN,False,0,0,False,False,0.0,TRamp,...,523.024414,71.332520,168.570435,-1689.796387,-263.968750,4225.609375,82907,76561198208074320,interz,10
829004,"[knife_butterfly, Glock-18, AK-47, C4]",0.00000,NaN,False,0,100,False,True,0.0,BombsiteB,...,-1987.031250,0.957184,-89.747314,398.769440,-159.968750,4225.609375,82907,76561198146207066,Boombl4,10
829005,"[knife_karambit, Glock-18, AWP, Molotov]",0.08080,0.0,False,0,100,False,True,0.0,Apartments,...,-1782.032715,6.216202,-36.306381,745.173157,-47.968750,4225.609375,82907,76561198055109028,ICY,10
829006,"[knife_karambit, Five-SeveN]",0.00910,0.0,False,0,100,False,False,0.0,Catwalk,...,-977.476196,23.986938,145.518036,-150.995728,-167.968750,4225.609375,82907,76561197991272318,ropz,10
829007,[],NaN,NaN,False,0,0,False,False,0.0,Apartments,...,-1646.583008,8.843994,-164.896545,692.394714,-47.968750,4225.609375,82907,76561198068422762,frozen,10
829008,"[knife_karambit, USP-S, MP9]",0.00000,NaN,False,0,75,False,False,0.0,TSpawn,...,1227.168091,9.702301,-119.256935,173.520187,-163.968750,4225.609375,82907,76561197989430253,karrigan,10
829009,"[knife_widowmaker, Glock-18, AK-47]",0.00641,0.0,False,0,100,False,True,0.0,BombsiteB,...,-2347.487793,-0.272949,-75.847206,-27.283997,-163.651428,4225.609375,82907,76561198068002993,HeavyGod,10


In [8]:
dem.kills[dem.kills['round'] == 10]

,tick,assistedflash,dmg_health,dmg_armor,attackerblind,headshot,hitgroup,noscope,penetrated,thrusmoke,...,victim_ping,victim_team_name,victim_team_clan_name,victim_name,victim_steamid,round,ticks_since_round_start,ticks_since_freeze_time_end,ticks_since_bomb_plant,clock
66,79057,False,27,3,False,False,None,False,0,False,...,0,CT,FaZe Clan,frozen,76561198068422762,10,2411,1131,<NA>,01:38
67,82026,False,175,6,False,True,None,False,0,False,...,0,TERRORIST,Cloud 9,interz,76561198208074320,10,5380,4100,<NA>,00:51
68,82539,True,27,3,False,False,None,False,0,False,...,0,CT,FaZe Clan,broky,76561198201620490,10,5893,4613,<NA>,00:43


### examine round 10

In [9]:
from awpy.plot import gif, PLOT_SETTINGS
from tqdm import tqdm
import os

if not os.path.isfile("de_mirage.gif"):
    frames = []

    for tick in tqdm(dem.ticks[dem.ticks["round"] == 10].tick.values[::128]):
        frame_df = dem.ticks[dem.ticks["tick"] == tick]
        frame_df = frame_df[
            ["X", "Y", "Z", "health", "armor_value", "pitch", "yaw", "team_name", "name"]
        ]

        points = []
        point_settings = []

        for _, row in frame_df.iterrows():
            points.append((row["X"], row["Y"], row["Z"]))

            # Determine team and corresponding settings
            team = "ct" if row["team_name"] == "CT" else "t"
            settings = PLOT_SETTINGS[team].copy()

            # Add additional settings
            settings.update(
                {
                    "hp": row["health"],
                    "armor": row["armor_value"],
                    "direction": (row["pitch"], row["yaw"]),
                    "label": row["name"],
                }
            )

            point_settings.append(settings)

        frames.append({"points": points, "point_settings": point_settings})

    print("Finished processing frames. Creating gif...")
    gif(f"{dem.header['map_name']}", frames, f"{dem.header['map_name']}.gif", duration=100)

# Round Durations

## What is the average duration of the rounds?

## How many players achieved multi-kills?